Chapter 2 Derivatives and Gradients
========

Example 2.1. Symbolic differentiation provides analytical derivatives.
----

In [6]:
import sympy as sp

In [7]:
x = sp.symbols('x')
f = x**2 + x/2 - sp.sin(x)/x
f_prime = sp.diff(f, x)

In [8]:
print("Function:")
sp.pprint(f)
print("\nDerivative:")
sp.pprint(f_prime)

Function:
 2   x   sin(x)
x  + ─ - ──────
     2     x   

Derivative:
      1   cos(x)   sin(x)
2⋅x + ─ - ────── + ──────
      2     x         2  
                     x   


2.3.1 Finite Difference Methods
----

In [9]:
import math

In [10]:
def diff_forward(f, x, h=1e-9):
    return (f(x + h) - f(x)) / h

def diff_central(f, x, h=1e-9):
    return (f(x + h/2) - f(x - h/2)) / h

def diff_backward(f, x, h=1e-9):
    return (f(x) - f(x - h)) / h

In [19]:
# Differentiate a sample function
f = math.sin
x = 1.0
h = 0.1  # Use a larger h to see differences

# Calculate derivatives 
print(f"Forward:  {diff_forward(f, x=x, h=h):.4f}")
print(f"Central:  {diff_central(f, x=x, h=h):.4f}")
print(f"Backward: {diff_backward(f, x=x, h=h):.4f}")
print(f"True:     {math.cos(x== 1.0):.4f}")

Forward:  0.4974
Central:  0.5401
Backward: 0.5814
True:     0.5403
